# Exploratory Data Analysis of Book Readers

## Getting acquainted with the data

Before diving into the exercises, let's see the structure of the data in the database.

In [1]:
import os
import pandas as pd
from sqlalchemy import *
from IPython.display import display
from source.utils import *

def as_generator(iterator):
    return (x for x in iterator)

def format_results(engine, query):
    result_set = engine.execute(query)
    return pd.DataFrame(as_generator(result_set), columns=result_set.keys())

psql = create_engine('postgresql://postgres:{}@db/postgres'.format(os.environ['POSTGRES_PASSWORD']))

metadata = MetaData()
metadata.reflect(bind=psql)
for table in metadata.sorted_tables:
    print(table.name)
    for column in table.columns:
        print('- {} {}'.format(column.name, column.type))
    print('-' * 4)

    
print("\nDenormalized data:")
format_results(psql,
               """
               SELECT r.visit_id AS reader_id, title, category_one, category_two, country, timezone,
                      location_accuracy, tracking_time, created_at
               FROM reading r INNER JOIN visits v ON r.visit_id = v.visitor_id
               INNER JOIN stories s ON r.story_id = s.id
               ORDER BY r.visit_id, r.tracking_time
               LIMIT 5
               """)

reading
- is_app_event BOOLEAN
- visitor_id CHAR(36)
- id CHAR(36)
- visit_id VARCHAR(100)
- tracking_time TIMESTAMP WITHOUT TIME ZONE
- created_at TIMESTAMP WITHOUT TIME ZONE
- story_id INTEGER
- user_id INTEGER
----
stories
- id INTEGER
- user_id INTEGER
- teaser VARCHAR(500)
- title VARCHAR(100)
- cover CHAR(36)
- category_one VARCHAR(20)
- category_two VARCHAR(20)
----
visits
- visitor_id CHAR(36)
- user_id INTEGER
- country VARCHAR(60)
- timezone VARCHAR(100)
- location_accuracy INTEGER
----

Denormalized data:


,reader_id,title,category_one,category_two,country,timezone,location_accuracy,tracking_time,created_at
0,004f9a0a-8235-4190-a336-cfe6fde1d14a,EXE,horror,other,United Kingdom,Europe/London,20,2016-02-21 20:41:58.178989,2016-02-21 20:41:58.178989
1,004f9a0a-8235-4190-a336-cfe6fde1d14a,EXE,horror,other,United Kingdom,Europe/London,20,2016-02-21 20:42:25.071397,2016-02-21 20:42:25.071397
2,004f9a0a-8235-4190-a336-cfe6fde1d14a,EXE,horror,other,United Kingdom,Europe/London,20,2016-02-21 20:42:41.061626,2016-02-21 20:42:41.061626
3,004f9a0a-8235-4190-a336-cfe6fde1d14a,EXE,horror,other,United Kingdom,Europe/London,20,2016-02-21 20:43:17.151303,2016-02-21 20:43:17.151303
4,004f9a0a-8235-4190-a336-cfe6fde1d14a,EXE,horror,other,United Kingdom,Europe/London,20,2016-02-21 20:43:34.135824,2016-02-21 20:43:34.135824


We can get to the real questions now.

## 1. How much reading is done by horror readers each day?
I'm assuming that Horror Readers are those who read horror stories but can also read other genres. Because of that, I'm taking into account stories read by horror readers that are not in the horror genre.

**IMPORTANT** Dear reviewer, please refer to [my notes](#Notes) at the end of the notebook for remarks about the data.

In [2]:
import pandas as pd
from source.utils import *

def as_generator(iterator):
    return (x for x in iterator)

def format_results(engine, query):
    result_set = engine.execute(query)
    return pd.DataFrame(as_generator(result_set), columns=result_set.keys())

### 1.1 SQL

In [3]:
# We have to account for the fact that horror readers can also read other genres
n1_sql = format_results(psql,
                        """
                        SELECT date_trunc(\'day\', tracking_time) AS date, COUNT(id) AS reads
                        FROM reading
                        WHERE reading.visit_id IN (
                            SELECT DISTINCT r.visit_id
                            FROM reading r INNER JOIN stories s ON r.story_id = s.id
                            WHERE s.category_one = \'horror\' OR s.category_two = \'horror\'
                        )
                        GROUP BY date_trunc(\'day\', reading.tracking_time)
                        ORDER BY date
                        """)

display(n1_sql)

,date,reads
0,2015-08-11,2
1,2015-08-13,1
2,2015-11-10,1
3,2015-11-25,5
4,2016-01-30,14
5,2016-02-15,1
6,2016-02-21,7
7,2016-02-23,4
8,2016-03-02,1
9,2016-04-17,11


### 1.2 Pandas

In [4]:
def first(arr):
    return arr[0]

def truncate_to_day(timestamp):
    return first(timestamp.split('T'))

visits = pd.read_csv('data/visits.csv')
reading = pd.read_csv('data/reading.csv')
stories = pd.read_csv('data/stories.csv')

reading.tracking_time = reading.tracking_time.apply(truncate_to_day)
reading.created_at = reading.created_at.apply(truncate_to_day)

horror_stories = stories[(stories.category_one == 'horror') | (stories.category_two == 'horror')]
horror_readers = pd.merge(left=reading, right=horror_stories, left_on='story_id', right_on='id').visit_id

horror_reading = reading[reading.visit_id.isin(horror_readers)]
horror_reading_per_day = horror_reading.groupby('tracking_time').size()

n1_pandas = pd.DataFrame({'date': horror_reading_per_day.index, 'reads': horror_reading_per_day.values})
display(n1_pandas)

,date,reads
0,2015-08-11,2
1,2015-08-13,1
2,2015-11-10,1
3,2015-11-25,5
4,2016-01-30,14
5,2016-02-15,1
6,2016-02-21,7
7,2016-02-23,4
8,2016-03-02,1
9,2016-04-17,11


### 1.3 Testing
Let's compare the results of both methods, side-by-side.

In [5]:
pd.concat([n1_sql, n1_pandas], axis=1)

,date,reads,date,reads
0,2015-08-11,2,2015-08-11,2
1,2015-08-13,1,2015-08-13,1
2,2015-11-10,1,2015-11-10,1
3,2015-11-25,5,2015-11-25,5
4,2016-01-30,14,2016-01-30,14
5,2016-02-15,1,2016-02-15,1
6,2016-02-21,7,2016-02-21,7
7,2016-02-23,4,2016-02-23,4
8,2016-03-02,1,2016-03-02,1
9,2016-04-17,11,2016-04-17,11


## 2. How many horror readers are active each day?
Again, I'm assuming that Horror Readers are those who read horror stories and other stuff too. Reading a story of another genre would also count as an activity for the day.

### 2.1 SQL
First, we can calculate the total number of horror readers.

In [6]:
total_horror_readers = \
    psql.execute("""
                SELECT COUNT(DISTINCT r.visit_id)
                FROM reading r INNER JOIN stories s ON r.story_id = s.id
                WHERE s.category_one = \'horror\' OR s.category_two = \'horror\'
                """)

print("Number of horror readers: {}".format(first(total_horror_readers.first())))

Number of horror readers: 43


Now, for the real question.

In [7]:
# We have to account for the fact that horror readers can also read other genres
n2_sql = format_results(psql,
                """
                SELECT date_trunc(\'day\', reading.tracking_time) AS date, COUNT(DISTINCT visit_id) AS unique_readers
                FROM reading
                WHERE reading.visit_id IN (
                    SELECT DISTINCT r.visit_id
                    FROM reading r INNER JOIN stories s ON r.story_id = s.id
                    WHERE s.category_one = \'horror\' OR s.category_two = \'horror\'
                )
                GROUP BY date_trunc(\'day\', reading.tracking_time)
                ORDER BY date
                """)

display(n2_sql)

,date,unique_readers
0,2015-08-11,1
1,2015-08-13,1
2,2015-11-10,1
3,2015-11-25,1
4,2016-01-30,1
5,2016-02-15,1
6,2016-02-21,1
7,2016-02-23,1
8,2016-03-02,1
9,2016-04-17,1


#### Remarks
Interestingly, the sum of daily active readers is **43**, the same as the total horror readers, meaning that no horror reader was active in two different days. My best hypothesis is that the the data does not contain tracking information that can follow the user between sessions. This would cause `reading.visit_id` to change each time the user starts a new session and the results would be similar to what we saw in the data.

### 2.2 Pandas

In [8]:
def count_unique(x):
    return len(set(x))

grouped_by_day = horror_reading.groupby('tracking_time')
unique_readers_per_day = grouped_by_day.visit_id.agg(count_unique)

n2_pandas = pd.DataFrame({'date': unique_readers_per_day.index, 'unique_readers': unique_readers_per_day.values})
display(n2_pandas)

,date,unique_readers
0,2015-08-11,1
1,2015-08-13,1
2,2015-11-10,1
3,2015-11-25,1
4,2016-01-30,1
5,2016-02-15,1
6,2016-02-21,1
7,2016-02-23,1
8,2016-03-02,1
9,2016-04-17,1


### 2.3 Testing
Let's compare the results of both methods, side-by-side.

In [9]:
pd.concat([n2_sql, n2_pandas], axis=1)

,date,unique_readers,date,unique_readers
0,2015-08-11,1,2015-08-11,1
1,2015-08-13,1,2015-08-13,1
2,2015-11-10,1,2015-11-10,1
3,2015-11-25,1,2015-11-25,1
4,2016-01-30,1,2016-01-30,1
5,2016-02-15,1,2016-02-15,1
6,2016-02-21,1,2016-02-21,1
7,2016-02-23,1,2016-02-23,1
8,2016-03-02,1,2016-03-02,1
9,2016-04-17,1,2016-04-17,1


## 3. What countries are the readers coming from?
### 3.1 SQL
First, an overall look at readers by country.

In [10]:
format_results(psql,
              """
              SELECT country, COUNT(DISTINCT visit_id) AS unique_readers
              FROM reading INNER JOIN visits ON reading.visit_id = visits.visitor_id
              WHERE reading.visit_id IN (
                SELECT DISTINCT r.visit_id
                FROM reading r INNER JOIN stories s ON r.story_id = s.id
                WHERE s.category_one = \'horror\' OR s.category_two = \'horror\'
              )
              GROUP BY country
              ORDER BY unique_readers DESC
              """)

,country,unique_readers
0,United States,23
1,United Kingdom,9
2,Canada,3
3,Australia,2
4,France,2
5,Philippines,1
6,Romania,1
7,Israel,1
8,Bangladesh,1


Where are the Brazilians? 🤔

Now, to answer the question posed, we can do:

In [11]:
n3_sql = format_results(psql,
              """
              SELECT date_trunc(\'day\', reading.tracking_time) AS date, country, COUNT(DISTINCT visit_id) AS unique_readers
              FROM reading INNER JOIN visits ON reading.visit_id = visits.visitor_id
              WHERE reading.visit_id IN (
                SELECT DISTINCT r.visit_id
                FROM reading r INNER JOIN stories s ON r.story_id = s.id
                WHERE s.category_one = \'horror\' OR s.category_two = \'horror\'
              )
              GROUP BY date_trunc(\'day\', reading.tracking_time), country
              ORDER BY date, country
              """)

display(n3_sql)

,date,country,unique_readers
0,2015-08-11,United States,1
1,2015-08-13,United States,1
2,2015-11-10,United States,1
3,2015-11-25,United States,1
4,2016-01-30,United Kingdom,1
5,2016-02-15,United Kingdom,1
6,2016-02-21,United Kingdom,1
7,2016-02-23,United States,1
8,2016-03-02,United States,1
9,2016-04-17,United Kingdom,1


### 3.2 Pandas

In [12]:
def transpose(list_tuples):
    return list(zip(*list_tuples))

grouped_by_day_and_country = pd.merge(left=horror_reading, right=visits, left_on='visit_id', right_on='visitor_id') \
                                .groupby(['tracking_time', 'country'])
                                      
unique_readers_per_day_country = grouped_by_day_and_country.visit_id.agg(count_unique)
dates, countries = transpose(unique_readers_per_day_country.index)

n3_pandas = pd.DataFrame({'date': dates,
                          'country': countries,
                          'unique_readers': unique_readers_per_day_country.values})

display(n3_pandas)

,country,date,unique_readers
0,United States,2015-08-11,1
1,United States,2015-08-13,1
2,United States,2015-11-10,1
3,United States,2015-11-25,1
4,United Kingdom,2016-01-30,1
5,United Kingdom,2016-02-15,1
6,United Kingdom,2016-02-21,1
7,United States,2016-02-23,1
8,United States,2016-03-02,1
9,United Kingdom,2016-04-17,1


### 3.3 Testing
Let's compare the results of both methods, side-by-side.

In [13]:
pd.concat([n3_sql, n3_pandas], axis=1)

,date,country,unique_readers,country,date,unique_readers
0,2015-08-11,United States,1,United States,2015-08-11,1
1,2015-08-13,United States,1,United States,2015-08-13,1
2,2015-11-10,United States,1,United States,2015-11-10,1
3,2015-11-25,United States,1,United States,2015-11-25,1
4,2016-01-30,United Kingdom,1,United Kingdom,2016-01-30,1
5,2016-02-15,United Kingdom,1,United Kingdom,2016-02-15,1
6,2016-02-21,United Kingdom,1,United Kingdom,2016-02-21,1
7,2016-02-23,United States,1,United States,2016-02-23,1
8,2016-03-02,United States,1,United States,2016-03-02,1
9,2016-04-17,United Kingdom,1,United Kingdom,2016-04-17,1


## Extracting geographic information from teasers
For this section, we are going to use a technique called [Named-Entity Recognition](https://en.wikipedia.org/wiki/Named-entity_recognition) to retrieve location information contained in the teasers.

But, first, let me demonstrate how it works with the example below:

In [14]:
from source.ner import NERTagger

sample_sentence = "Rafael is interviewing with Inkitt, a company located in Berlin that helps writers publish books"
tagger = NERTagger()
tagger.tag_sentence(sample_sentence)

[('Rafael', 'PERSON'),
 ('is', 'O'),
 ('interviewing', 'O'),
 ('with', 'O'),
 ('Inkitt', 'ORGANIZATION'),
 (',', 'O'),
 ('a', 'O'),
 ('company', 'O'),
 ('located', 'O'),
 ('in', 'O'),
 ('Berlin', 'LOCATION'),
 ('that', 'O'),
 ('helps', 'O'),
 ('writers', 'O'),
 ('publish', 'O'),
 ('books', 'O')]

The sentence is broken into words and each word receive a class. Words with the O class are considered to have "no class". For our text mining activity, we have to look out for words tagged with LOCATION.

Let's do this!

In [15]:
def extract_locations(sentence):
    if not isinstance(sentence, str):
        return ''
        
    def locations():
        tagged = tagger.tag_sentence(sentence)
        for word, tag in tagged:
            if tag == 'LOCATION':
                yield word
    
    print(".", end='')
    return ','.join(locations())


stories['locations'] = stories.teaser.apply(extract_locations)
display(stories)

....

,id,user_id,teaser,title,cover,category_one,category_two,locations
0,18965,5242,"A story of Horror, a story of Tragedy. Pain, v...",EXE,d69810df7ed140322435505f81cad7eb.jpg,horror,other,
1,18966,19465,NaN,Untitled,NaN,NaN,NaN,
2,18967,18598,John and Amarildo were the best of friends gro...,My Best Friend,895e1732b870e88f79f302fbb814bc72.jpg,horror,drama,
3,18968,19465,NaN,Untitled,NaN,NaN,NaN,
4,18969,19507,A MET detective investigates a mysterious part...,The Party,5054c0f875c89a69a153d225c2912d42.jpg,fantasy,mystery,London
5,18970,19511,What starts out as a simple assassin's contrac...,Flint & Tinder Book One: A March Of White,0fc3e2872fcab659ccee5186982baaf1.jpg,fantasy,mystery,


One of the entries contained location information but the `stories` dataset consists of only 6 entries... not much data to validate our solution. How about using a scraper to get more teasers from the website?

In [16]:
from source.scraping import books_at

books = pd.DataFrame(list(flatmap(books_at, range(1, 3))))

print("\nNumber of books retrived: {}".format(len(books)))
display(books.head())

Scraping results at page 1
Scraping results at page 2

Number of books retrived: 40


,author,description,genre,title
0,ChicagoDreams,He tightly clenched his jaw as he stared at me...,Romance / Humor,His Dangerous Ways
1,Kat Halstead,Pop Princess Chloe Cole was the absolute last ...,Romance / Drama,Need Me Like That
2,Caitlin Andrews,I just sat there letting the man I called fath...,Other / Fantasy,My Muted Mate
3,SilverCrystal006921,"Book 1 of The Phoenix King Trilogy, 'The Awake...",Fantasy / Adventure,The Phoenix King: The Awakening [Book 1]
4,Kimiko Hamada,A loving father who only wants to do right by ...,Romance / Humor,Balancing A Sweet Romance


That's more like it. Now, we go over the same procedure as before to extract the locations.

In [17]:
books['locations'] = books.description.apply(extract_locations)

........................................

Which books have locations in their teasers, according to our model?

In [18]:
books[books.locations != '']

,author,description,genre,title,locations
35,Vanessa Ravencroft,Roy and his alien pet are now on their way to ...,Scifi / Adventure,Galaxy at War,Saresii
37,India Millar,Berengaria. Richard the Lionheart's wife. The ...,Romance / Other,The Queen that Time Forgot,England


### Limitations
The model appears to have problems with teasers from fictional stories. Some examples:

In [19]:
print(tagger.tag_sentence(books.iloc[22].description))
print('-' * 4)
print(tagger.tag_sentence(books.iloc[30].description))
print('-' * 4)
print(tagger.tag_sentence(books.iloc[35].description))

[('A', 'O'), ('spaceship', 'O'), ('disaster', 'O'), ('forces', 'O'), ('the', 'O'), ('teenage', 'O'), ('daughter', 'O'), ('of', 'O'), ('the', 'O'), ('Antemonian', 'O'), ('High', 'O'), ('Warlord', 'O'), ('to', 'O'), ('crash-land', 'O'), ('on', 'O'), ('the', 'O'), ('Terran', 'O'), ('colony', 'O'), ('of', 'O'), ('Geos', 'O'), (',', 'O'), ('only', 'O'), ('a', 'O'), ('few', 'O'), ('years', 'O'), ('after', 'O'), ('an', 'O'), ('uneasy', 'O'), ('truce', 'O'), ('ends', 'O'), ('outright', 'O'), ('war', 'O'), ('.', 'O'), ('Can', 'O'), ('she', 'O'), ('get', 'O'), ('home', 'O'), ('?', 'O')]
----
[('With', 'O'), ('one', 'O'), ('awkward', 'O'), ('semester', 'O'), ('under', 'O'), ('her', 'O'), ('belt', 'O'), (',', 'O'), ('Kallima', 'PERSON'), ('returns', 'O'), ('to', 'O'), ('the', 'O'), ('Fairy', 'O'), ('Realm', 'O'), ('with', 'O'), ('high', 'O'), ('hopes', 'O'), ('.', 'O'), ('Unfortunately', 'O'), (',', 'O'), ('a', 'O'), ('plot', 'O'), ('unfolds', 'O'), (',', 'O'), ('pitting', 'O'), ('her', 'O'), ('fr

On the last example, the model almost got it right.

## Notes
Looking at the columns `tracking_time` and `created_at` of table `reading`, the values appear to be always the same, Upon further inspection, we can see that this is not always the case, though. Probably, `tracking_time` registers the time when the reading really happened and `created_at` registers the time of inserting into the data, as `created_at` >= `tracking_time` on the data. Events are generated when the user scrolls down while reading a story, and contain the following information:
```JSON
{
    "events": [
        {
            "id": "c0e778e7-522d-47d8-a726-0b4e2ca1fedd",
            "name": "Reading Chapter",
            "properties": {
                "storyId": 18965,
                "chapterNumber": 2,
                "from": {}
            },
            "time": 1509148130.507
        }
    ],
    "visit_token": "ca96c8c6-9a51-4bb2-8b14-4f1a5575f18f",
    "visitor_token": "06509f38-4325-45d4-af65-4a8c585e904b"
}
```

Still on `reading`, it isn't clear which of the fields holds the identity of the reader. It could be either `visitor_id` or `user_id`, judging by the names, but both of them have problems. Inspecting the `visits` table, we can see the same `user_id` associated with **multiples values for `country`**, which would very unlikely in a real setting. On the other side, `visitor_id` cannot be used because there is no overlap between values of `reading.visitor_id` and `visits.visitor_id`, rendering a `JOIN` impossible. However, all is not lost: we can join the tables using `reading.visit_id` and `visit.visitor_id`, which was my choice for the test. See queries below.

In [20]:
print("Joining on 'visitor_id' in both tables")
display(format_results(psql, "SELECT * FROM reading r INNER JOIN visits v ON r.visitor_id=v.visitor_id"))
print("\n")
print("Joining on 'reading.visit_id' and 'visits.visitor_id'")
display(format_results(psql, "SELECT * FROM reading r INNER JOIN visits v ON r.visit_id=v.visitor_id LIMIT 3"))

Joining on 'visitor_id' in both tables


,is_app_event,visitor_id,id,visit_id,tracking_time,created_at,story_id,user_id,visitor_id,user_id,country,timezone,location_accuracy




Joining on 'reading.visit_id' and 'visits.visitor_id'


,is_app_event,visitor_id,id,visit_id,tracking_time,created_at,story_id,user_id,visitor_id,user_id,country,timezone,location_accuracy
0,False,22ac4ffa-9ba4-4eb5-a027-48bc4fd67ef2,e855b15a-c0dd-4b5f-a82b-642ce6f32bcb,1ff3c16e-fc20-4319-8720-f64be2079f90,2017-06-29 21:49:49.490028,2017-06-29 21:49:49.492249,18965,None,1ff3c16e-fc20-4319-8720-f64be2079f90,None,United States,America/Los_Angeles,20
1,False,22ac4ffa-9ba4-4eb5-a027-48bc4fd67ef2,5a1f4e84-0398-4343-a427-6fe37a12d5fa,1ff3c16e-fc20-4319-8720-f64be2079f90,2017-06-29 21:50:08.757155,2017-06-29 21:50:08.759166,18965,None,1ff3c16e-fc20-4319-8720-f64be2079f90,None,United States,America/Los_Angeles,20
2,False,22ac4ffa-9ba4-4eb5-a027-48bc4fd67ef2,76de6153-1cf3-42f3-9760-66640dab3625,1ff3c16e-fc20-4319-8720-f64be2079f90,2017-06-29 21:50:14.990401,2017-06-29 21:50:14.993954,18965,None,1ff3c16e-fc20-4319-8720-f64be2079f90,None,United States,America/Los_Angeles,20
